In [1]:
!pip install biopython fuc pybedtools
!apt-get install bedtools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 56.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 99.0 MB/s eta 0:00:00
  Created wheel for pybedtools: filename=pybedtools-0.12.0-cp311-cp311-linux_x86_64.whl size=14261434 sha256=470785561f2cff9fecd952d5b898b0cbc3acd1aef918aaba908d991edb38632d
  Stored in directory: /root/.cache/pip/wheels/95/62/57/e8c47c2a43884227b6644cf1d2dabf3162a0f40b9c1493c1f5
Successfully built pybedtools
Readi

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Data import CodonTable
from Bio import Entrez
from Bio.SeqFeature import SeqFeature, FeatureLocation
import pandas as pd
import pybedtools
import re
from fuc import pybed

In [3]:
scaffolds = {}
for scaffold in SeqIO.parse('/content/GCF_943734665.1_idAnoAquaMG_Q_19_genomic.fna', format='fasta'):
    scaffolds[scaffold.id] = scaffold

# Получение протеома

In [136]:
annotation_df = pd.read_csv('genomic.gff', sep='\t', comment='#', header=None, names=[
    "scaffold", "source", "type", "start", "end", "score", "strand", "phase", "attributes"
])
cds_df = annotation_df[annotation_df['type'] == 'CDS'].reset_index()

In [137]:
with open('proteins.fasta', 'w') as prot_file:
    cur_transcript=None
    cur_cds = Seq("")
    for index, row in cds_df.iterrows():
        attrs = row['attributes'].split(';')
        parent = attrs[1]
        gene = attrs[5]
        phase = 0
        if parent != cur_transcript:
            if index != 0:
                if strand == '-':
                    cur_cds = cur_cds.reverse_complement()
                protein_seq = cur_cds.translate(to_stop=True)
                prot_file.write(f'>{gene} scaffold={scaffold} strand={strand} {cur_transcript} \n')
                prot_file.write(f'{str(protein_seq)}\n')

            cur_transcript = parent
            cur_cds = Seq("")
            strand = row['strand']
            phase = int(row['phase'])
            scaffold = row['scaffold']

        beg = row['start'] - 1 + phase
        end = row['end']
        seq = scaffolds[scaffold].seq[beg:end]
        cur_cds = cur_cds + seq

    if strand == '-':
        cur_cds = cur_cds.reverse_complement()
    protein_seq = cur_cds.translate(to_stop=True)
    prot_file.write(f'> {scaffold} {strand} {cur_transcript}\n')
    prot_file.write(f'{str(protein_seq)}\n')

## Квадруплексы

In [138]:
PQS = []
PQS_minus = []
pattern="(?:G{3,5}[ATGC]{1,7}){3,}G{3,5}"
pattern_minus="(?:C{3,5}[ATGC]{1,7}){3,}C{3,5}"
for scaffold, record in scaffolds.items():
  name, sequence = record.id, str(record.seq)
  PQS += [[name, m.start(),m.end(),m.group(0), name, '+'] for m in re.finditer(pattern,sequence,re.IGNORECASE)]
for scaffold, record in scaffolds.items():
  name, sequence = record.id, str(record.seq)
  PQS_minus += [[name, m.start(),m.end(),m.group(0), name, '-'] for m in re.finditer(pattern_minus,sequence,re.IGNORECASE)]
print(len(PQS))
print(len(PQS_minus))

12803
12883


In [132]:
quadrs = pd.DataFrame(PQS+PQS_minus, columns=['Chromosome', 'Start', 'End', 'Sequence', 'Name', 'Strand'])

In [133]:
bf = pybed.BedFrame.from_frame(meta=[], data=quadrs)
bf.to_file('quadruplex.bed')

### Разбиение аннотации

In [134]:
with open('scaffolds_sizes.txt', 'w') as out:
    for scaffold in scaffolds:
        scaffold_name = scaffold
        scaffold_length = len(scaffolds[scaffold].seq)
        out.write(f"{scaffold_name}\t{scaffold_length}\n")

In [135]:
def add_introns(annotation_df):
    introns = []
    for gene in annotation_df[annotation_df['type'] == 'gene'].itertuples():
        print(gene)
        gene_exons = annotation_df[(annotation_df['type'] == 'exon') &
                                   (annotation_df['scaffold'] == gene.scaffold) &
                                   (annotation_df['start'] >= gene.start) &
                                   (annotation_df['end'] <= gene.end)]
        gene_exons = gene_exons.sort_values(by='start')
        previous_exon_end = None
        for exon in gene_exons.itertuples():
            if previous_exon_end is not None:
                intron_start = previous_exon_end + 1
                intron_end = exon.start - 1
                if intron_start < intron_end:
                    introns.append({
                        "scaffold": gene.scaffold,
                        "source": "predicted",
                        "type": "intron",
                        "start": intron_start,
                        "end": intron_end,
                        "score": ".",
                        "strand": gene.strand,
                        "phase": ".",
                        "attributes": gene.attributes
                    })
            previous_exon_end = exon.end

    introns_df = pd.DataFrame(introns)
    return pd.concat([annotation_df, introns_df], ignore_index=True)

annotation_intron_df = add_introns(annotation_df)

Pandas(Index=1, scaffold='NC_064876.1', start=971, end=6658, type='gene', name='ID=gene-LOC126572356;Dbxref=GeneID:126572356;Name=LOC126572356;description=uncharacterized', strand='-')


AttributeError: 'Pandas' object has no attribute 'attributes'

In [139]:
!awk '{if($1 !~ /^#/ ) print $1"\t"$4-1"\t"$5"\t"$3"\t"$9"\t"$7}' genomic.gff > annotation.bed

In [140]:
!head annotation.bed

NC_064876.1	0	8489435	region	ID=NC_064876.1:1..8489435;Dbxref=taxon:42839;Name=X;chromosome=X;gbkey=Src;genome=chromosome;mol_type=genomic	+
NC_064876.1	971	6658	gene	ID=gene-LOC126572356;Dbxref=GeneID:126572356;Name=LOC126572356;description=uncharacterized	-
NC_064876.1	971	6513	transcript	ID=rna-XR_007607957.1;Parent=gene-LOC126572356;Dbxref=GeneID:126572356,Genbank:XR_007607957.1;Name=XR_007607957.1;experiment=COORDINATES:	-
NC_064876.1	6387	6513	exon	ID=exon-XR_007607957.1-1;Parent=rna-XR_007607957.1;Dbxref=GeneID:126572356,Genbank:XR_007607957.1;experiment=COORDINATES:	-
NC_064876.1	1904	2044	exon	ID=exon-XR_007607957.1-2;Parent=rna-XR_007607957.1;Dbxref=GeneID:126572356,Genbank:XR_007607957.1;experiment=COORDINATES:	-
NC_064876.1	1696	1836	exon	ID=exon-XR_007607957.1-3;Parent=rna-XR_007607957.1;Dbxref=GeneID:126572356,Genbank:XR_007607957.1;experiment=COORDINATES:	-
NC_064876.1	1488	1628	exon	ID=exon-XR_007607957.1-4;Parent=rna-XR_007607957.1;Dbxref=GeneID:126572356,Genbank:XR_00

In [141]:
!awk '$4 == "exon"' annotation.bed > exons.bed
!sort -k1,1 -k2,2n exons.bed > exons_sorted.bed
!bedtools merge -i exons_sorted.bed -c 4,5,6 -o distinct > exons.bed
!head exons.bed

NC_064876.1	971	1212	exon	ID=exon-XM_050231609.1-3;Parent=rna-XM_050231609.1;Dbxref=GeneID:126572356,Genbank:XM_050231609.1;experiment=COORDINATES:,ID=exon-XM_050231617.1-6;Parent=rna-XM_050231617.1;Dbxref=GeneID:126572356,Genbank:XM_050231617.1;experiment=COORDINATES:,ID=exon-XM_050231625.1-6;Parent=rna-XM_050231625.1;Dbxref=GeneID:126572356,Genbank:XM_050231625.1;experiment=COORDINATES:,ID=exon-XM_050231634.1-5;Parent=rna-XM_050231634.1;Dbxref=GeneID:126572356,Genbank:XM_050231634.1;experiment=COORDINATES:,ID=exon-XR_007607957.1-6;Parent=rna-XR_007607957.1;Dbxref=GeneID:126572356,Genbank:XR_007607957.1;experiment=COORDINATES:	-
NC_064876.1	1243	1420	exon	ID=exon-XM_050231602.1-3;Parent=rna-XM_050231602.1;Dbxref=GeneID:126572356,Genbank:XM_050231602.1;gbkey=mRNA;gene=LOC126572356;product=uncharacterized,ID=exon-XM_050231617.1-5;Parent=rna-XM_050231617.1;Dbxref=GeneID:126572356,Genbank:XM_050231617.1;experiment=COORDINATES:,ID=exon-XM_050231625.1-5;Parent=rna-XM_050231625.1;Dbxref=Gene

In [145]:
! awk '$3 == "gene"' genomic.gtf > genes.gtf
! awk '$3 == "exon"' genomic.gtf > exons.gtf

! bedtools subtract -a genes.gtf -b exons.gtf > introns.gtf
! awk '$3 == "intron" {print $1 "\t" $4-1 "\t" $5 "\t" $9}' introns.gtf > introns.bed

In [146]:
annotation_intron_df[['scaffold', 'start', 'end', 'type', 'scaffold', 'strand']].to_csv('annotation.bed', header=None, index=False, sep='\t')

In [147]:
!awk '$4 == "intron"' annotation.bed > introns.bed
!sort -k1,1 -k2,2n introns.bed > introns_sorted.bed
!bedtools merge -i introns_sorted.bed -c 4,5,6 -o distinct > introns.bed
!head introns.bed

NC_064876.1	1213	1243	intron	NC_064876.1	-
NC_064876.1	1421	1488	intron	NC_064876.1	-
NC_064876.1	1629	1696	intron	NC_064876.1	-
NC_064876.1	1837	1904	intron	NC_064876.1	-
NC_064876.1	2045	2709	intron	NC_064876.1	-
NC_064876.1	2845	6001	intron	NC_064876.1	-
NC_064876.1	6266	6387	intron	NC_064876.1	+,-
NC_064876.1	6659	11641	intron	NC_064876.1	+
NC_064876.1	11872	12132	intron	NC_064876.1	+
NC_064876.1	12461	12518	intron	NC_064876.1	+


In [148]:
!awk '{if($1 !~ /^#/ ) print $1"\t"$4-1"\t"$5"\t"$3"\t"$9"\t"$7}' genomic.gff > annotation.bed

In [100]:
!head introns.bed

NC_064876.1	1213	1243	intron	NC_064876.1	-
NC_064876.1	1421	1488	intron	NC_064876.1	-
NC_064876.1	1629	1696	intron	NC_064876.1	-
NC_064876.1	1837	1904	intron	NC_064876.1	-
NC_064876.1	2045	2709	intron	NC_064876.1	-
NC_064876.1	2845	6001	intron	NC_064876.1	-
NC_064876.1	6266	6387	intron	NC_064876.1	+,-
NC_064876.1	6659	11641	intron	NC_064876.1	+
NC_064876.1	11872	12132	intron	NC_064876.1	+
NC_064876.1	12461	12518	intron	NC_064876.1	+


Гены

In [149]:
!awk 'BEGIN{OFS="\t"} {if ($4 == "gene") {print $1, $2, $3, $4, $5, $6}}' annotation.bed > genes.bed

In [150]:
!bedtools complement -i genes.bed -g scaffolds_sizes.txt > intergenic.bed

Промоторы

In [151]:
!bedtools flank -i genes.bed -g scaffolds_sizes.txt -l 1000 -r 0 -s > promoters.bed

Downstream

In [152]:
!bedtools flank -i genes.bed -g scaffolds_sizes.txt -l 0 -r 200 -s  > downstream.bed

### Пересечение с квадруплексами

In [177]:
!cat exons.bed | wc -l
!cat introns.bed | wc -l
!cat promoters.bed | wc -l
!cat downstream.bed | wc -l
!cat intergenic.bed | wc -l

60320
49547
12876
12874
10992


In [178]:
!bedtools intersect -a quadruplex.bed -b exons.bed -wa -s | wc -l
!bedtools intersect -a quadruplex.bed -b introns.bed -wa -s | wc -l
!bedtools intersect -a quadruplex.bed -b promoters.bed -wa -s | wc -l
!bedtools intersect -a quadruplex.bed -b downstream.bed -wa -s | wc -l
!bedtools intersect -a quadruplex.bed -b intergenic.bed -wa | wc -l

591
6858
574
186
9864


In [179]:
591 / 60320, 6858 / 49547, 574 / 12876, 186 / 12874, 9864 / 10992

(0.009797745358090186,
 0.13841403112196501,
 0.04457906182044113,
 0.014447724095075345,
 0.8973799126637555)

### Кол-во участков с квадруплексами

In [157]:
!bedtools intersect -a quadruplex.bed -b exons.bed -wb -s > exons_with_quadruplex_all.bed
!sort -u exons_with_quadruplex_all.bed > exons_with_quadr.bed
!cat exons_with_quadr.bed | wc -l
!cat exons.bed | wc -l
!echo
!bedtools intersect -a quadruplex.bed -b introns.bed -wb -s > introns_with_quadruplex_all.bed
!sort -u introns_with_quadruplex_all.bed > introns_with_quadr.bed
!cat introns_with_quadr.bed | wc -l
!cat introns.bed | wc -l
!echo
!bedtools intersect -a quadruplex.bed -b promoters.bed -wb -s > promoters_with_quadruplex_all.bed
!sort -u promoters_with_quadruplex_all.bed > promoters_with_quadr.bed
!cat promoters_with_quadr.bed | wc -l
!cat promoters.bed | wc -l
!echo
!bedtools intersect -a quadruplex.bed -b downstream.bed -wb -s > downstream_with_quadruplex_all.bed
!sort -u downstream_with_quadruplex_all.bed > downstream_with_quadr.bed
!cat downstream_with_quadr.bed | wc -l
!cat downstream.bed | wc -l
!echo
!bedtools intersect -a quadruplex.bed -b intergenic.bed -wb > intergenic_with_quadruplex_all.bed
!sort -u intergenic_with_quadruplex_all.bed > intergenic_with_quadr.bed
!cat intergenic_with_quadr.bed | wc -l
!cat intergenic.bed | wc -l

591
60320

6858
49547

574
12876

186
12874

9864
10992


## Z-Hunt

Я запускал z-hunt локально в несколько потоков для ускорения получения результата, сейчас лишь подргузим файл с результатом. В файле на этапе работы zhunt я уже оставлял только хиты с z-score > 400 дабы файл не нужно было очищать

### Преобразование в bed формат

In [158]:
zscore_file = 'zscore_backup_small'
chunks = pd.read_csv(zscore_file,
              skiprows=1,
              names=["start","end","3","4","5","score","seq","8"],
              delim_whitespace=True,
              chunksize=10000)
filtered_df = pd.concat(chunks, ignore_index=True)
zhunt_df = filtered_df

/tmp/ipython-input-158-758813473.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  chunks = pd.read_csv(zscore_file,


In [159]:
zhunt_sorted = zhunt_df.sort_values(by=['start'])
zhunt_sorted[1] = 'genome'
zhunt_sorted[[1, 'start', 'end', 'score', 'seq']].to_csv('zscore_sorted.bed', sep='\t', header=None, index=False)

In [160]:
!bedtools merge -i zscore_sorted.bed > zscore_merged.bed

In [161]:
zhunt_coords = pd.read_csv('zscore_merged.bed', sep='\t', names=['place', 'start', 'end'])

### Разметка по скаффолдам

In [162]:
annotation_df = pd.read_csv('annotation.bed', names=['scaffold', 'start', 'end', 'type', 'name', 'strand'], sep='\t', header=None)
region_df = annotation_df[annotation_df['type'] == 'region'].reset_index(drop=True)
region_df['global_end'] = region_df['end'].cumsum()

i_scaffold = 0
scaffold_start = 1
scaffold_end =  176590975
scaffold = 'NC_064876.1'

corrected_records = []

for index, row in zhunt_coords.iterrows():
    zdna_start = row['start']
    zdna_end = row['end']
    if zdna_start > scaffold_end:
        i_scaffold += 1
    if i_scaffold > 0:
        prev_cumulative_end = region_df.iloc[i_scaffold - 1]['global_end']
        scaffold_end = region_df.iloc[i_scaffold]['global_end']
        scaffold = region_df.iloc[i_scaffold]['scaffold']
    else:
        prev_cumulative_end = 0
    corrected_start = zdna_start - prev_cumulative_end
    corrected_end = zdna_end - prev_cumulative_end
    corrected_records.append({
            'scaffold': scaffold,
            'start': corrected_start,
            'end': corrected_end,
        })

corrected_zhunt_df = pd.DataFrame(corrected_records)
corrected_zhunt_df.head()

,scaffold,start,end
0,NC_064876.1,1207,1244
1,NC_064876.1,1415,1452
2,NC_064876.1,1623,1660
3,NC_064876.1,1831,1868
4,NC_064876.1,2039,2076


In [163]:
corrected_zhunt_df[[4, 5, 6]] = [4, 5, 6]
corrected_zhunt_df.to_csv('zhunt.bed', sep='\t', header=None, index=False)

### Результаты z-hunt в участках

In [180]:
!bedtools intersect -a zhunt.bed -b exons.bed -wa > zhunt_in_exons.bed
!cat zhunt_in_exons.bed | wc -l
!echo
!bedtools intersect -a zhunt.bed -b introns.bed -wa > zhunt_in_introns.bed
!cat zhunt_in_introns.bed | wc -l
!echo
!bedtools intersect -a zhunt.bed -b promoters.bed -wa > zhunt_in_promoters.bed
!cat zhunt_in_promoters.bed | wc -l
!echo
!bedtools intersect -a zhunt.bed -b downstream.bed -wa > zhunt_in_downstream.bed
!cat zhunt_in_downstream.bed | wc -l
!echo
!bedtools intersect -a zhunt.bed -b intergenic.bed -wa > zhunt_in_intergenic.bed
!cat zhunt_in_intergenic.bed | wc -l

2265

3687

746

113

5265


In [56]:
!cat zhunt.bed | wc -l

163286


In [181]:
2265 / 60320, 3687 / 49547, 746 / 12876, 113 / 12874, 5265 / 10992

(0.03754973474801061,
 0.07441419258481846,
 0.057937247592420006,
 0.008777380767438248,
 0.4789847161572052)

### Число участков с результатами Z-hunt

In [165]:
!bedtools intersect -a exons.bed -b zhunt.bed -wa > exons_with_zhunt_all.bed
!sort -u exons_with_zhunt_all.bed > exons_with_zhunt.bed
!cat exons_with_zhunt.bed | wc -l
!cat exons.bed | wc -l
!echo
!bedtools intersect -a introns.bed -b zhunt.bed -wa > introns_with_zhunt_all.bed
!sort -u introns_with_zhunt_all.bed > introns_with_zhunt.bed
!cat introns_with_zhunt.bed | wc -l
!cat introns.bed | wc -l
!echo
!bedtools intersect -a promoters.bed -b zhunt.bed -wa > promoters_with_zhunt_all.bed
!sort -u promoters_with_zhunt_all.bed > promoters_with_zhunt.bed
!cat promoters_with_zhunt.bed | wc -l
!cat promoters.bed | wc -l
!echo
!bedtools intersect -a downstream.bed -b zhunt.bed -wa > downstream_with_zhunt_all.bed
!sort -u downstream_with_zhunt_all.bed > downstream_with_zhunt.bed
!cat downstream_with_zhunt.bed | wc -l
!cat downstream.bed | wc -l
!echo
!bedtools intersect -a intergenic.bed -b zhunt.bed -wa > intergenic_with_zhunt_all.bed
!sort -u intergenic_with_zhunt_all.bed > intergenic_with_zhunt.bed
!cat intergenic_with_zhunt.bed | wc -l
!cat intergenic.bed | wc -l

1150
60320

742
49547

385
12876

101
12874

299
10992


In [62]:
!cat zhunt.bed | wc -l

163286


In [182]:
1150 / 60320, 742 / 49547, 385 / 12876, 101 / 12874, 299 /10992

(0.019064986737400532,
 0.01497567965769875,
 0.029900590245417833,
 0.007845269535497903,
 0.02720160116448326)

## Z-DNABERT

In [ ]:
with open("text_predictions.txt", "r") as preds:
  with open("zdnabert.bed", "w") as bed:
    chrom = 'NC_064876.1'
    for line in preds.readlines():
      if 'GCF' in line:
        continue
      if 'start' in line:
        continue
      if 'NC_' in line or 'NW' in line:
        chrom = line.strip()
        print(chrom)
        continue
      try:
        start, end = map(int, line.split())
        bed.write(chrom + "\t" + str(start) + "\t" + str(end) + "\n")
      except:
        continue


In [184]:
!bedtools intersect -a zdnabert.bed -b exons.bed -wa > zdna_in_exons.bed
!cat zdna_in_exons.bed | wc -l
!echo
!bedtools intersect -a zdnabert.bed -b introns.bed -wa | wc -l
!echo
!bedtools intersect -a zdnabert.bed -b promoters.bed -wa | wc -l
!echo
!bedtools intersect -a zdnabert.bed -b downstream.bed -wa | wc -l
!echo
!bedtools intersect -a zdnabert.bed -b intergenic.bed -wa | wc -l

9461

14937

3262

353

13993


In [188]:
!bedtools intersect -a zdnabert.bed -b intergenic.bed -wa | wc -l

13993


In [189]:
9461 / 60320, 14937 / 49547, 3262 / 12876, 353 / 12874, 13993 /10992

(0.15684681697612732,
 0.3014713302520839,
 0.2533395464429947,
 0.027419605406245145,
 1.2730167394468705)

In [124]:
9447 / 37986, 14937 / 37986, 3262 / 37986, 353 / 37986, 13993 / 37986

(0.24869688832727846,
 0.3932238193018481,
 0.08587374295793188,
 0.009292897383246458,
 0.36837255831095667)

In [190]:
!bedtools intersect -a exons.bed -b zdnabert.bed -wa > exons_with_zdnabert_all.bed
!sort -u exons_with_zdnabert_all.bed > exons_with_zdnabert.bed
!cat exons_with_zdnabert.bed | wc -l
!cat exons.bed | wc -l
!echo
!bedtools intersect -a introns.bed -b zdnabert.bed -wa > introns_with_zdnabert_all.bed
!sort -u introns_with_zdnabert_all.bed > introns_with_zdnabert.bed
!cat introns_with_zdnabert.bed | wc -l
!cat introns.bed | wc -l
!echo
!bedtools intersect -a promoters.bed -b zdnabert.bed -wa > promoters_with_zdnabert_all.bed
!sort -u promoters_with_zdnabert_all.bed > promoters_with_zdnabert.bed
!cat promoters_with_zdnabert.bed | wc -l
!cat promoters.bed | wc -l
!echo
!bedtools intersect -a downstream.bed -b zdnabert.bed -wa > downstream_with_zdnabert_all.bed
!sort -u downstream_with_zdnabert_all.bed > downstream_with_zdnabert.bed
!cat downstream_with_zdnabert.bed | wc -l
!cat downstream.bed | wc -l
!echo
!bedtools intersect -a intergenic.bed -b zdnabert.bed -wa > intergenic_with_zdnabert_all.bed
!sort -u intergenic_with_zdnabert_all.bed > intergenic_with_zdnabert.bed
!cat intergenic_with_zdnabert.bed | wc -l
!cat intergenic.bed | wc -l

4397
60320

2774
49547

2166
12876

325
12874

1742
10992


In [191]:
4397 / 60320, 2774 / 49547, 2166 / 12876, 325 / 12874, 1742 /10992

(0.07289456233421751,
 0.05598724443457727,
 0.16821994408201305,
 0.02524467919838434,
 0.1584788937409025)

## Фоновое распределение

In [111]:
genome_length =  176590975

In [192]:
!cat exons.bed | wc -l
!cat introns.bed | wc -l
!cat promoters.bed | wc -l
!cat downstream.bed | wc -l
!cat intergenic.bed | wc -l

60320
49547
12876
12874
10992


In [117]:
total = 60360 + 49547 + 12876 + 12874 + 10992

In [193]:
60360 / total

0.4115950330380705

In [119]:
49547 / total

0.33786115145688006

In [120]:
12876 / total

0.08780148517889655

In [121]:
12874 / total

0.08778784717250032

In [122]:
10992 / total

0.07495448315365259

## Распределения хитов

### Квадруплексы

In [170]:
quadrs = pd.read_csv('quadruplex.bed', sep='\t', header=None)
quadrs['len'] = quadrs[2] - quadrs[1]
len_quadrs = quadrs['len'].sum()
print(len_quadrs)

611355


In [171]:
len_quadrs / genome_length

np.float64(0.00346198326386725)

### Результаты z-hunt

In [172]:
zhunt_coords['len'] = zhunt_coords['end'] - zhunt_coords['start']
len_zhunt = zhunt_coords['len'].sum()
print(len_zhunt)

4517716


In [173]:
len_zhunt / genome_length

np.float64(0.025582938199418176)

### Результаты Z-DNABERT

In [174]:
bert = pd.read_csv('zdnabert.bed', sep='\t', header=None)
bert['len'] = bert[2] - bert[1]
len_bert = bert['len'].sum()
print(len_bert)

646927


In [175]:
len_bert / genome_length

np.float64(0.003663420511722074)